# Investments results inspection
Routines for converting, inspecting and visualizing _pommesinvest_ investment results.
* Plot investment decisions
* Plot decomissionings
* Plot total installed capacity
* Include exogenous capacities to evaluate plants status

## Package imports

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pommesevaluation.investment_results_inspection import (
    preprocess_raw_results, 
    aggregate_investment_results, 
    extract_data_and_save,
    plot_single_investment_variable, 
    plot_single_investment_variable_for_all_cases,
)

## Script parameterization
* Define model configuration / results to be analyzed
* Define column order and colors for evaluation
* Read in the actual results

In [ ]:
# Model configuration
time_frame_in_years = 26
frequency = "1H"
dr_scenario = "none"
dr_scenarios = ["none", "5", "50", "95"]
sensitivities = ["pv_-50%", "pv_+50%", "prices_-50%", "prices_+50%", "consumption_-50%", "consumption_+50%"]
fuel_price_scenario = "NZE_high"
emissions_pathway = "long-term"
impose_investment_maxima = False

# Paths, filenames and color codes
path_results = "./model_results/pommesinvest/"
path_processed_data = "./data_out/"
path_plots = "./plots/"

FUELS_EXISTING = {
    "uranium": "#e50000",
    "lignite": "#7f2b0a",
    "hardcoal": "#000000",
    "mixedfuels": "#a57e52",
    "otherfossil": "#d8dcd6",
}

FUELS = {
    "biomass": "#15b01a",
    "hydrogen": "#6fa8dc",
    "natgas": "#ffd966",
    "oil": "#aaa662",
    "waste": "#c04e01"
}

STORAGES_INFLOW = {
    "PHS_inflow": "#0c2aac",
    "battery_inflow": "#f7e09a"
}
STORAGES_OTHER = {
    "PHS_outflow": "#0c2aac",
    "battery_outflow": "#f7e09a",
    "PHS_capacity": "#0c2aac",
    "battery_capacity": "#f7e09a"
}

DEMAND_RESPONSE_CLUSTERS = {
    "hoho_cluster_shift_only": "#333333", 
    "hoho_cluster_shift_shed": "#555555", 
    "ind_cluster_shed_only": "#666666",
    "ind_cluster_shift_only": "#888888", 
    "ind_cluster_shift_shed": "#aaaaaa", 
    "tcs+hoho_cluster_shift_only": "#cccccc",
    "tcs_cluster_shift_only": "#dddddd", 
}

filename = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{frequency}"
)
if impose_investment_maxima:
    annual_investment_limits = ""
else:
    annual_investment_limits = "_no_annual_limit"
if dr_scenario != "none":
    file_add_on = (
        f"_with_dr_{dr_scenario}_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
    )
else:
    file_add_on = (
        f"_no_dr_50_"
        f"fuel_price-{fuel_price_scenario}_"
        f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
    )
file_extension = ".csv"

filename_backup = "installed_capacity_backup_generation.csv"
filename_storage = "installed_storage_capacity.csv"

plt.rcParams.update({'font.size': 12})

# Control workflow
include_storages_in_plots = True
aggregation_mode = "energy_carrier"

# Single scenario analyses
Inspect the results for a single scenario model run.
## Read in, preprocess and aggregate data
See imported functions for documentation

In [ ]:
investment_results_raw = pd.read_csv(f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0)
processed_results = preprocess_raw_results(investment_results_raw)
aggregated_results, other_storage_results = aggregate_investment_results(
    processed_results, energy_carriers=FUELS, by="energy_carrier"
)

## Visualize results
Results visualization is separated between investments in storage energy content as well as storage outflow power and the remainder, i.e. investments in generation units, investments in storage inflow power and investments in demand response clusters.

The following parameters are displayed
* Newly installed capacity
* Decommissioned capacity
* Total installed capacity

### Generation, storage inflow and demand response

In [ ]:
if include_storages_in_plots:
    if dr_scenario != "none":
        colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
    else:
        colors = {**FUELS, **STORAGES_INFLOW}
else:
    if dr_scenario != "none":
        colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}
    else:
        colors = FUELS

plot_single_investment_variable(
    aggregated_results,
    "invest",
    colors=colors,
    aggregation=aggregation_mode,
    save=True,
    filename="results_invest_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data,
    ylim=[0, 50000]
)

In [ ]:
if include_storages_in_plots:
    if dr_scenario != "none":
        colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
    else:
        colors = {**FUELS, **STORAGES_INFLOW}
else:
    if dr_scenario != "none":
        colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}
    else:
        colors = FUELS

plot_single_investment_variable(
    aggregated_results,
    "old",
    colors=colors,
    aggregation=aggregation_mode,
    save=True,
    filename="results_old_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data
)

In [ ]:
if include_storages_in_plots:
    if dr_scenario != "none":
        colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
    else:
        colors = {**FUELS, **STORAGES_INFLOW}
else:
    if dr_scenario != "none":
        colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}
    else:
        colors = FUELS

plot_single_investment_variable(
    aggregated_results,
    "total",
    colors=colors,
    aggregation=aggregation_mode,
    save=True,
    filename="results_total_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data
)

### Storage outflow and storage energy content

In [ ]:
plot_single_investment_variable(
    other_storage_results,
    "invest",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True,
    save=True,
    filename="results_storages_invest_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data
)

In [ ]:
plot_single_investment_variable(
    other_storage_results,
    "old",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True,
    save=True,
    filename="results_storages_old_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data
)

In [ ]:
plot_single_investment_variable(
    other_storage_results,
    "total",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True,
    save=True,
    filename="results_storages_total_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data
)

## Include exogenous capacities in assessment
* Read in capacities of backup plants, aggregated by fuel
* Read in installed power and capacity of storage units

In [ ]:
backup_generation = pd.read_csv(
    f"{path_processed_data}{filename_backup}", index_col=0
)
if time_frame_in_years <= 26:
    backup_generation = backup_generation.loc[:2045]
backup_generation.index = backup_generation.index.astype(str)

In [ ]:
grouped_total = aggregated_results[["total"]].reset_index()
grouped_total = grouped_total.pivot(
    index="energy_carrier", columns="year", values="total"
).T

In [ ]:
overall_capacity = grouped_total.add(backup_generation)
overall_capacity.loc[:,overall_capacity.isna().all()] = backup_generation
overall_capacity.loc[:,overall_capacity.isna().all()] = grouped_total

In [ ]:
if include_storages_in_plots:
    if dr_scenario != "none":
        colors = {**FUELS_EXISTING, **FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
    else:
        colors = {**FUELS_EXISTING, **FUELS, **STORAGES_INFLOW}
else:
    if dr_scenario != "none":
        colors = {**FUELS_EXISTING, **FUELS, **DEMAND_RESPONSE_CLUSTERS}
    else:
        colors = {**FUELS_EXISTING, **FUELS}

plot_single_investment_variable(
    overall_capacity.T,
    "all",
    colors=colors,
    group=False,
    save=True,
    filename="results_all_dr_scenario",
    dr_scenario=dr_scenario,
    path_plots=path_plots,
    path_data_out=path_processed_data,
    ylim=[0, 200000]
)

In [ ]:
if dr_scenario != "none":
    overall_capacity["demand_response_overall"] = overall_capacity[[
        "hoho_cluster_shift_only", 
        "hoho_cluster_shift_shed", 
        "ind_cluster_shed_only",
        "ind_cluster_shift_only",
        "ind_cluster_shift_shed",
        "tcs+hoho_cluster_shift_only",
        "tcs_cluster_shift_only"
    ]].sum(axis=1)

In [ ]:
overall_capacity

# Multi scenario analyses
Inspect and compare the results among scenarios.

## Read in, preprocess and aggregate data

In [ ]:
investment_results_raw = {}
aggregated_results = {}
other_storage_results = {}
for dr_scenario in dr_scenarios:
    if dr_scenario != "none":
        file_add_on = (
            f"_with_dr_{dr_scenario}_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
        )
    else:
        file_add_on = (
            f"_no_dr_50_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
        )
    investment_results_raw[dr_scenario] = (
        pd.read_csv(f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0)
    )
    processed_results = preprocess_raw_results(investment_results_raw[dr_scenario])
    aggregated_results[dr_scenario], other_storage_results[dr_scenario] = aggregate_investment_results(
        processed_results, energy_carriers=FUELS, by="energy_carrier"
    )

## Visualize results
### Generation, storage inflow and demand response

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
else:
    colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable_for_all_cases(
    aggregated_results,
    "invest",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=DEMAND_RESPONSE_CLUSTERS,
    save=True,
    filename="results_invest",
    path_plots=path_plots,
    ylim=(0, 50000),
)

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
else:
    colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable_for_all_cases(
    aggregated_results,
    "old",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=DEMAND_RESPONSE_CLUSTERS,
    save=True,
    filename="results_old",
    path_plots=path_plots,
    ylim=(0, 50000),
)

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
else:
    colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable_for_all_cases(
    aggregated_results,
    "total",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=DEMAND_RESPONSE_CLUSTERS,
    save=True,
    filename="results_total",
    path_plots=path_plots,
    ylim=(0, 300000),
)

### Storage outflow and storage energy content

In [ ]:
plot_single_investment_variable_for_all_cases(
    other_storage_results,
    "invest",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True,
    dr_color_codes={},
    save=True,
    filename="results_storages_invest",
    path_plots=path_plots,
)

In [ ]:
plot_single_investment_variable_for_all_cases(
    other_storage_results,
    "old",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True,
    dr_color_codes={},
    save=True,
    filename="results_storages_old",
    path_plots=path_plots,
)

In [ ]:
plot_single_investment_variable_for_all_cases(
    other_storage_results,
    "total",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True,
    dr_color_codes={},
    save=True,
    filename="results_storages_total",
    path_plots=path_plots,
)

## Include exogenous capacities in assessment

In [ ]:
overall_capacity = {}
for dr_scenario in dr_scenarios:
    grouped_total = aggregated_results[dr_scenario][["total"]].reset_index()
    grouped_total = grouped_total.pivot(
        index="energy_carrier", columns="year", values="total"
    ).T
    overall_capacity[dr_scenario] = grouped_total.add(backup_generation)
    overall_capacity[dr_scenario].loc[:, overall_capacity[dr_scenario].isna().all()] = backup_generation
    overall_capacity[dr_scenario].loc[:, overall_capacity[dr_scenario].isna().all()] = grouped_total
    overall_capacity[dr_scenario] = overall_capacity[dr_scenario].T

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS_EXISTING, **FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}

else:
    colors = {**FUELS_EXISTING, **FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable_for_all_cases(
    overall_capacity,
    "all",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=DEMAND_RESPONSE_CLUSTERS,
    group=False,
    save=True,
    filename="results_all",
    path_plots=path_plots,
    ylim=(0, 350000),
)

In [ ]:
for dr_scenario in dr_scenarios:
    if dr_scenario != "none":
        overall_capacity[dr_scenario].loc["demand_response_overall"] = overall_capacity[dr_scenario].loc[[
            "hoho_cluster_shift_only", 
            "hoho_cluster_shift_shed", 
            "ind_cluster_shed_only",
            "ind_cluster_shift_only",
            "ind_cluster_shift_shed",
            "tcs+hoho_cluster_shift_only",
            "tcs_cluster_shift_only"
        ]].sum()
    overall_capacity[dr_scenario].to_csv(f"{path_processed_data}overall_capacity_dr_scenario_{dr_scenario}.csv", sep=";")

# Analyses of sensitivities
Inspect and compare among the sensitivities considered for scenario DR 50

## Read in, preprocess and aggregate data

In [ ]:
investment_results_raw = {}
aggregated_results = {}
other_storage_results = {}
file_add_on = (
    f"_with_dr_50_"
    f"fuel_price-{fuel_price_scenario}_"
    f"co2_price-{emissions_pathway}{annual_investment_limits}_"
)
renamed_sensitivities = []

for sensitivity in sensitivities:
    renamed_sensitivity = f"{sensitivity.split('_')[0]} {sensitivity.split('_')[-1]}"
    renamed_sensitivities.append(renamed_sensitivity)
    file_add_on_sensitivity = f"sensitivity_{sensitivity}_investment"
    
    investment_results_raw[sensitivity] = (
        pd.read_csv(f"{path_results}{filename}{file_add_on}{file_add_on_sensitivity}{file_extension}", index_col=0)
    )
    processed_results = preprocess_raw_results(investment_results_raw[sensitivity])
    aggregated_results[renamed_sensitivity], other_storage_results[sensitivity] = aggregate_investment_results(
        processed_results, energy_carriers=FUELS, by="energy_carrier"
    )

## Visualize results
### Generation, storage inflow and demand response

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
else:
    colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable_for_all_cases(
    aggregated_results,
    "invest",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=DEMAND_RESPONSE_CLUSTERS,
    save=True,
    filename="results_invest_sensitivities",
    path_plots=path_plots,
    ylim=(0, 50000),
    title="Demand Reponse scenario 50; sensitivity:",
)

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
else:
    colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable_for_all_cases(
    aggregated_results,
    "old",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=DEMAND_RESPONSE_CLUSTERS,
    save=True,
    filename="results_old_sensitivities",
    path_plots=path_plots,
    ylim=(0, 50000),
    title="Demand Reponse scenario 50; sensitivity:",
)

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}
else:
    colors = {**FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable_for_all_cases(
    aggregated_results,
    "total",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=DEMAND_RESPONSE_CLUSTERS,
    save=True,
    filename="results_total_sensitivities",
    path_plots=path_plots,
    ylim=(0, 300000),
    title="Demand Reponse scenario 50; sensitivity:",
)

### Storage outflow and storage energy content

In [ ]:
plot_single_investment_variable_for_all_cases(
    other_storage_results,
    "invest",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True,
    dr_color_codes={},
    save=True,
    filename="results_storages_invest_sensitivities",
    path_plots=path_plots,
    title="Demand Reponse scenario 50; sensitivity:",
)

In [ ]:
plot_single_investment_variable_for_all_cases(
    other_storage_results,
    "old",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True,
    dr_color_codes={},
    save=True,
    filename="results_storages_old_sensitivities",
    path_plots=path_plots,
    title="Demand Reponse scenario 50; sensitivity:",
)

In [ ]:
plot_single_investment_variable_for_all_cases(
    other_storage_results,
    "total",
    colors=STORAGES_OTHER,
    aggregation="energy_carrier",
    storage=True,
    dr_color_codes={},
    save=True,
    filename="results_storages_total_sensitivities",
    path_plots=path_plots,
    title="Demand Reponse scenario 50; sensitivity:",
)

## Include exogenous capacities in assessment

In [ ]:
overall_capacity = {}
for sensitivity in renamed_sensitivities:
    grouped_total = aggregated_results[sensitivity][["total"]].reset_index()
    grouped_total = grouped_total.pivot(
        index="energy_carrier", columns="year", values="total"
    ).T
    overall_capacity[sensitivity] = grouped_total.add(backup_generation)
    overall_capacity[sensitivity].loc[:, overall_capacity[sensitivity].isna().all()] = backup_generation
    overall_capacity[sensitivity].loc[:, overall_capacity[sensitivity].isna().all()] = grouped_total
    overall_capacity[sensitivity] = overall_capacity[sensitivity].T

In [ ]:
if include_storages_in_plots:
    colors = {**FUELS_EXISTING, **FUELS, **STORAGES_INFLOW, **DEMAND_RESPONSE_CLUSTERS}

else:
    colors = {**FUELS_EXISTING, **FUELS, **DEMAND_RESPONSE_CLUSTERS}

plot_single_investment_variable_for_all_cases(
    overall_capacity,
    "all",
    colors=colors,
    aggregation="energy_carrier",
    dr_color_codes=DEMAND_RESPONSE_CLUSTERS,
    group=False,
    save=True,
    filename="results_all_sensitivities",
    path_plots=path_plots,
    ylim=(0, 350000),
    title="Demand Reponse scenario 50; sensitivity:",
)

In [ ]:
for sensitivity in renamed_sensitivities:
    overall_capacity[sensitivity].loc["demand_response_overall"] = overall_capacity[sensitivity].loc[[
        "hoho_cluster_shift_only", 
        "hoho_cluster_shift_shed", 
        "ind_cluster_shed_only",
        "ind_cluster_shift_only",
        "ind_cluster_shift_shed",
        "tcs+hoho_cluster_shift_only",
        "tcs_cluster_shift_only"
    ]].sum()
    overall_capacity[sensitivity].to_csv(f"{path_processed_data}overall_capacity_dr_scenario_50_sensitivity_{sensitivity.split(' ')[0]}_{sensitivity.split(' ')[1]}.csv", sep=";")

# Preparation of results for usage in AMIRIS
> _**Note**:_
>
> * _The following section creates the necessary outputs that can be passed to the AMIRIS converter notebook._
> * _Data is extracted and saved but not visualized._
> * _Data aggregation differs compared to the remainder of this notebook._

Extract data for all scenarios and save respective outputs to file.

In [ ]:
for dr_scen in dr_scenarios:
    if dr_scen != "none":
        file_add_on = (
            f"_with_dr_{dr_scen}_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
        )
    else:
        file_add_on = (
            f"_no_dr_50_"
            f"fuel_price-{fuel_price_scenario}_"
            f"co2_price-{emissions_pathway}{annual_investment_limits}_investment"
        )
    investment_results_raw = (
        pd.read_csv(f"{path_results}{filename}{file_add_on}{file_extension}", index_col=0)
    )
    processed_results = preprocess_raw_results(investment_results_raw)
    aggregated_results, other_storage_results = aggregate_investment_results(
        processed_results, 
        energy_carriers=FUELS, 
        by=["energy_carrier", "technology"], 
        include_chp_information=True
    )    
    
    for variable_name in ["invest", "old", "total"]:
        extract_data_and_save(
            aggregated_results,
            variable_name,
            aggregation=["energy_carrier", "technology"],
            group=True,
            filename=f"results_for_amiris_{variable_name}_dr_scenario",
            dr_scenario=dr_scen,
            path_data_out="./data_out/",
        )